In [23]:
import pandas as pd
import numpy as np

dfs=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

df=dfs[0]

df=df.replace('Not assigned',np.nan)

df.dropna(subset=['Borough'],inplace=True)

df.fillna('',inplace=True)

df['Neighbourhood']=df.groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ','.join(x))
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True)
df.drop(['index'],axis=1,inplace=True)
df['Neighbourhood']=df['Neighbourhood'].replace('',df['Borough'])
df=df.rename(columns={"Postcode":"Postal Code"})

In [24]:
df.shape

(103, 3)

In [25]:
geo=pd.read_csv('http://cocl.us/Geospatial_data')

In [47]:
data=pd.concat([df.set_index(['Postal Code']),geo.set_index(['Postal Code'])], axis=1, sort=False)

In [48]:
data

,Borough,Neighbourhood,Latitude,Longitude
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
M7A,Queen's Park,Queen's Park,43.662301,-79.389494
M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M3B,North York,Don Mills North,43.745906,-79.352188
M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
